In [ ]:
'''
REFERENCES: https://github.com/areed1192/sigma_coding_youtube/tree/master/python/python-finance/td-ameritrade

'''
import urllib                       #parse info for access token etc...
import json                         #content back
import requests                     #API component
import dateutil.parser              #Date Parser - take info from first api request and parse it
import datetime                     
from TDAuth import TDAuthentication
from config import password, account_number, client_id
import pandas as pd
import csv

TDClient = TDAuthentication(client_id, account_number, password)
TDClient.authenticate()

access_token = TDClient.access_token


In [2]:
def unix_time_millis(dt):
    
    # grab the starting point, so time '0'
    epoch = datetime.datetime.utcfromtimestamp(0)
    
    return (dt - epoch).total_seconds() * 1000.0



In [48]:
#PRICE HISTORY - CHANGE PRODUCT NAME - OUTPUTS LAST MONTH OF OHLCV DATA!

product = "SPX"

endpoint = r"https://api.tdameritrade.com/v1/marketdata/{}/pricehistory".format(product)

payload = {'apikey': client_id,
           #'period':'2',
           #'periodType':'day',
           'frequencyType':'minute',
           'frequency':'1',
           'startDate': 1549024201000,
           'endDate': 1572916805000,
           'needExtendedHoursData':'true'
          }

content = requests.get(url=endpoint, params  = payload)

#convert it to dict

data = content.json()
#print (data)
print('Done 1')


#data_pd = json.loads(data)

#print (data['candles'])
#str(product)+
with open(str(product)+(str(".csv")), "w", newline='') as writeFile:
    f = csv.writer(writeFile)
    f.writerow([ 'Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume'])

    for this_data in data['candles']:
        unix_times = int(this_data['datetime']) / 1000


        actual_date = datetime.datetime.fromtimestamp(unix_times).strftime('%Y-%m-%d')
        actual_time = datetime.datetime.fromtimestamp(unix_times).strftime('%H:%M')
        f.writerow( [ actual_date, actual_time, this_data['open'], this_data['high'], this_data['low'], this_data['close'], this_data['volume'] ] )

writeFile.close()

#print(data)

print("Done 2")

Done 1
Done 2


In [90]:
#OPTIONS HISTORY - CHANGE PRODUCT AND FROMDATE / TODATE

# need to convert json object to csv file tomorrow.

product = "SPY"

endpoint = "https://api.tdameritrade.com/v1/marketdata/chains"

# get our access token
headers = {'Authorization': "Bearer {}".format(access_token)}

# this endpoint, requires fields which are separated by ','
params = {'apikey': client_id,
           'symbol': product,
           'contractType':'ALL',
           'strikeCount': 5,
           'includeQuotes': 'TRUE',
           'range' : 'NTM',
           'fromDate' : '2019-11-01',
           'toDate' : '2020-02-01',
           'daysToExpiration' : 100,
           'expMonth' : 'ALL',
           'optionType': 'ALL'
          }

# make a request
content = requests.get(url = endpoint, params = params, headers = headers)

#convert it to dict

data = content.json()

print("DONE 2")

{'symbol': 'SPY', 'status': 'SUCCESS', 'underlying': {'symbol': 'SPY', 'description': 'SPDR S&P 500', 'change': 0.88, 'percentChange': 0.29, 'close': 298.4, 'quoteTime': 1571356815461, 'tradeTime': 1571356800001, 'bid': 299.1, 'ask': 299.16, 'last': 299.28, 'mark': 299.28, 'markChange': 0.88, 'markPercentChange': 0.29, 'bidSize': 100, 'askSize': 1400, 'highPrice': 300.24, 'lowPrice': 298.52, 'openPrice': 299.68, 'totalVolume': 46784885, 'exchangeName': 'PAC', 'fiftyTwoWeekHigh': 302.63, 'fiftyTwoWeekLow': 233.76, 'delayed': False}, 'strategy': 'SINGLE', 'interval': 0.0, 'isDelayed': False, 'isIndex': False, 'interestRate': 2.42788, 'underlyingPrice': 299.13, 'volatility': 29.0, 'daysToExpiration': 0.0, 'numberOfContracts': 140, 'callExpDateMap': {'2019-11-01:14': {'298.0': [{'putCall': 'CALL', 'symbol': 'SPY_110119C298', 'description': 'SPY Nov 1 2019 298 Call (Weekly)', 'exchangeName': 'OPR', 'bid': 3.78, 'ask': 3.81, 'last': 3.77, 'mark': 3.8, 'bidSize': 40, 'askSize': 40, 'bidAskSiz

In [12]:
#REAL TIME QUOTES

endpoint = r"https://api.tdameritrade.com/v1/marketdata/{}/quotes".format('//ES')

payload = {'apikey': client_id
          }

content = requests.get(url=endpoint, params  = payload)

#convert it to dict
data = content.json()

data

{'ES': {'assetType': 'EQUITY',
  'symbol': 'ES',
  'description': 'Eversource Energy (D/B/A) Common Stock',
  'bidPrice': 84.0,
  'bidSize': 1200,
  'bidId': 'P',
  'askPrice': 85.3,
  'askSize': 0,
  'askId': ' ',
  'lastPrice': 85.12,
  'lastSize': 91100,
  'lastId': 'N',
  'openPrice': 84.54,
  'highPrice': 85.33,
  'lowPrice': 84.47,
  'bidTick': ' ',
  'closePrice': 84.58,
  'netChange': 0.54,
  'totalVolume': 1106148,
  'quoteTimeInLong': 1571345603570,
  'tradeTimeInLong': 1571351400010,
  'mark': 85.12,
  'exchange': 'n',
  'exchangeName': 'NYSE',
  'marginable': True,
  'shortable': True,
  'volatility': 0.0121,
  'digits': 2,
  '52WkHigh': 86.26,
  '52WkLow': 61.09,
  'nAV': 0.0,
  'peRatio': 30.9066,
  'divAmount': 2.14,
  'divYield': 2.53,
  'divDate': '2019-09-19 00:00:00.000',
  'securityStatus': 'Normal',
  'regularMarketLastPrice': 85.12,
  'regularMarketLastSize': 911,
  'regularMarketNetChange': 0.54,
  'regularMarketTradeTimeInLong': 1571351400010,
  'netPercentChang